In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import matplotlib.mlab as mlab

from scipy.stats import norm
import tabulate as tabulate

import yfinance as yf

In [228]:
#find daily returns
tickers = ['AAPL', 'IBM', 'GOOGL', 'BP', 'XOM', 'COST', 'GS']
weights = np.array([0.15,0.2,0.2,0.15,0.1,0.15,0.05])
initial_investment = 1000000

df = yf.download(tickers, '2016-01-01', '2016-12-31')['Close']
percentage = df.pct_change().dropna()

ret = percentage
cov = percentage.cov()
meanReturns = ret.mean()

Time = 365
ret['portfolio'] = ret.dot(weights)
ret

[*********************100%***********************]  7 of 7 completed


,AAPL,BP,COST,GOOGL,GS,IBM,XOM,portfolio
Date,,,,,,,,
2016-01-04,0.000855,-0.006398,-0.012136,-0.023869,-0.017145,-0.012135,-0.006286,-0.011008
2016-01-05,-0.025059,-0.004185,0.002445,0.002752,-0.017218,-0.000736,0.008521,-0.005100
2016-01-06,-0.019570,-0.020369,-0.009254,-0.002889,-0.024413,-0.005006,-0.008321,-0.012901
2016-01-07,-0.042205,-0.028713,-0.022909,-0.024140,-0.030735,-0.017090,-0.016006,-0.026713
2016-01-08,0.005288,-0.017669,-0.017504,-0.013617,-0.004131,-0.009258,-0.020202,-0.011096
...,...,...,...,...,...,...,...,...
2016-12-23,0.001978,0.002980,0.000062,-0.002322,0.003540,-0.002095,-0.001761,0.000508
2016-12-27,0.006351,0.002431,-0.000185,0.002637,0.002448,0.002579,0.000441,0.002451
2016-12-28,-0.004264,0.003773,-0.006418,-0.006618,-0.003767,-0.005684,-0.004959,-0.003639


In [229]:
#portfolio performance
def ptperformance(weights, mean, cov, Time):
    Ret = np.sum(mean*weights)*Time
    std= np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(Time)
    return Ret, std



#Find the percentile of distribution at 95% confidence level.
def historicalVar(returns, alpha=5):
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalVar, alpha=5)
    else:
        raise TypeError("not a series or dataframe")
        
Var = -historicalVar(ret['portfolio'], alpha=5)*np.sqrt(Time)
print("Historical Var at 95 CI = " + str(round(Var*initial_investment,2)))

#find expectation of all values below VaR
def historicalCVar(returns, alpha=5):
    if isinstance(returns, pd.Series):
        belowvar = returns <= historicalVar(returns, alpha=alpha)
        return returns[belowvar].mean()
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalCVar, alpha=5)
    else:
        raise TypeError("not a series or dataframe")
CVar = -historicalCVar(ret['portfolio'], alpha=5)*np.sqrt(Time)
print("Historical CVar at 95 CI = " + str(round(CVar*initial_investment,2)))

Historical Var at 95 CI = 282622.33
Historical CVar at 95 CI = 422927.56


In [230]:
#covariance method - assume normal dist
def var_param(ptret, ptsd, alpha=5, dof=6):
    return norm.ppf(1-(alpha/100))*ptsd - ptret
   
def cvar_param(ptret, ptsd, alpha=5, dof=6):
    cvar =  (alpha/100)**-1 *norm.pdf(norm.ppf(alpha/100))*ptsd - ptret
    return cvar

pRet, pStd = ptperformance(weights, meanReturns, cov, Time)

normVar =  var_param(pRet, pStd)
normCVar = cvar_param(pRet, pStd)

print("Normal dist Var at 95 CI = " + str(round(normVar*initial_investment,2)))
print("Normal dist CVar at 95 CI = " + str(round(normCVar*initial_investment,2)))

Normal dist Var at 95 CI = 91377.0
Normal dist CVar at 95 CI = 166665.37
